In [3]:
import pandas as pd
import sqlalchemy as sal

In [4]:
sal.__version__

'1.4.39'

SQLAlchelmy è una libreria di Python per interagire con database relazionali (SQL, MySQL, PostgreSQL etc) tramite la componente Object Relational Mapper. L'ORM permette di mappare classi Python e oggetti Python in tabelle relazionali ed entità all'interno di un DB.
Qual è il vantaggio nell'utilizzare un ORM? \
 1. Utilizzare un ORM ci permette di essere svincolati dal dialetto specifico del DB una volta stabilita la specifica connessione.
 2. l'esecuzione di query sulle tabelle o le operazioni sul DB è indipendente dal dialetto specifico del DB,
 3. Un ORM è molto pratico in fase di creazione perchè possiamo generare tabelle e definirne i metadati mappandoli come classi python.


Svolgeremo l'esempio in cui vogliamo creare un DB locale definendone struttura e tabelle, inserendone i dati ed eseguendo query su di essi:

Per indirizzarci al DB dobbiamo inizializzare un oggetto della classe *engine*. \
Attraverso l'engine possiamo settare i parametri (credenziali utente, dialetto, indirizzo etc) che definiscono la connessione verso il DB tramite la *connection string* \
documentazione: https://docs.sqlalchemy.org/en/20/core/engines.html \
Se il DB non esiste verrà automaticamente creato. Nel nostro esempio creiamo un semplice DB sqlite:

In [3]:
from sqlalchemy import create_engine

In [4]:
engine = create_engine('sqlite:///miodb.sqlite', echo=True) 


La prima cosa da fare è costruire il DB Schema o in termin di sqlAlchemy, costruire una classe che definisce le tabelle del DB, o per meglio dire mappare la tabella ad una classe descrivendone i vari attributi.

Per mappare una tabella in SQLAlchemy bisogna innanzitutto definire la classe *Base* dalla quale ereditare gli attributi che permetteno la creazione della tabella.
[https://docs.sqlalchemy.org/en/13/orm/extensions/declarative/basic_use.html]

In [5]:
#from sqlalchemy.orm import declarative_base #oppure
from sqlalchemy.ext.declarative import declarative_base

In [6]:
Base = declarative_base()

Chiamando declarative_base() otteniamo una nuova classe base da cui erediteranno le classi-tabelle che definiamo.
Quando una classe-tabella viene definita saranno automaticmaente generati una Table ed un mapper(), strumenti essenziali per l'effettiva costruzione.

Per mappare una tabella dobbiamo importare da SQLAlchemy le classi che ci permettono di definire i tipi di dati contenuti all'interno della tabella. \
Innanzitutto bisogna importare la classe Column che ci permette di definire le colonne che andiamo a creare nella tabella e a seguire i loro datatype:

In [7]:
from sqlalchemy import Column, String, DateTime, Integer, CHAR, ForeignKey
from datetime import datetime

In [2]:
class Pokemon(Base): #eredita dalla classe Base
    __tablename__ = "pokemon" #definiamo il nome tabella con il dundermethod specifico
    
    #__table_args__ = {'extend_existing': True} 
    id_pkm = Column(
        Integer(), 
        primary_key=True #permette di assegnare la caratteristica di primary-key
    )
    nome = Column(
        String(25), #possiamo chiamare la classe del datatype per assegnare il valore massimo dell'attributo
        nullable=False)#fa sì che l'attributo non possa essere nullo
    
    tipo = Column(
        String(25), #possiamo chiamare la classe del datatype per assegnare il valore massimo dell'attributo
        nullable=False) #fa sì che l'attributo non possa essere nullo
    
    generazione = Column(
        Integer, 
        nullable=False)
    
    data_creazione = Column(
        DateTime(),
        default=datetime.utcnow) #valore di default da assegnare in fase di creazione dell'attributo
    
    def __init__(self, id_pkm, nome, tipo, generazione):
        self.id = id_pkm
        self.nome = nome
        self.tipo = tipo
        self.generazione = generazione
        #self.data = data
        
    
    def __repr__(self): #definiamo come vogliamo rappresentare l'oggetto della classe quando viene visualizzato
        return f"({self.id}) {self.nome}, {self.tipo}, {self.generazione})"
    

NameError: name 'Base' is not defined

In [9]:
class Debolezze(Base):
    __tablename__ = "debolezze"
    
    d_id = Column("d_id", Integer, primary_key=True)
    tipo = Column("tipo", String, ForeignKey("pokemon.tipo"))
    debolezza = Column("debolezza", String)
                  
    def __init__(self, d_id, tipo, debolezza):
        self.id = d_id
        self.tipo = tipo
        self.debolezza = debolezza
    
    def __rpr__(self):
        return f"({self.id}, il tipo {self.tipo} è debole al tipo {self.debolezza})"

In [10]:
#Mostriamo i metadati della nostra tabella:
Pokemon.__table__

Table('pokemon', MetaData(), Column('id_pkm', Integer(), table=<pokemon>, primary_key=True, nullable=False), Column('nome', String(length=25), table=<pokemon>, nullable=False), Column('tipo', String(length=25), table=<pokemon>, nullable=False), Column('generazione', Integer(), table=<pokemon>, nullable=False), Column('data_creazione', DateTime(), table=<pokemon>, default=ColumnDefault(<function datetime.utcnow at 0x7f99b192a7a0>)), schema=None)

In [11]:
Debolezze.__table__

Table('debolezze', MetaData(), Column('d_id', Integer(), table=<debolezze>, primary_key=True, nullable=False), Column('tipo', String(), ForeignKey('pokemon.tipo'), table=<debolezze>), Column('debolezza', String(), table=<debolezze>), schema=None)

In [12]:
Base.metadata.create_all(bind=engine)

2022-11-24 17:15:38,202 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-24 17:15:38,203 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("pokemon")
2022-11-24 17:15:38,203 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-24 17:15:38,204 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("pokemon")
2022-11-24 17:15:38,204 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-24 17:15:38,205 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("debolezze")
2022-11-24 17:15:38,205 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-24 17:15:38,205 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("debolezze")
2022-11-24 17:15:38,206 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-24 17:15:38,206 INFO sqlalchemy.engine.Engine 
CREATE TABLE pokemon (
	id_pkm INTEGER NOT NULL, 
	nome VARCHAR(25) NOT NULL, 
	tipo VARCHAR(25) NOT NULL, 
	generazione INTEGER NOT NULL, 
	data_creazione DATETIME, 
	PRIMARY KEY (id_pkm)
)


2022-11-24 17:15:38,207 INFO sqlalchemy.engine.Engine [no

Come possiamo verificare dall'output generato trmaite il parametro "echo=True" la tabella Utenti è stata creata all'interno del database.

Inseriamo dati all'interno della tabella: \
Per interagire con le tablele del DB dobbiamo avviare una sesssione.
La sessione stabilisce le relazione e la comunicazione con il DB e rappresenta una staging area per tutti gli oggetti ORM da mappare, i dati magari da inserire o le query da eseguire.
Proprio come con GIT, le informazioni inserite nella connessione devono essere "committate".


In [13]:
from sqlalchemy.orm import sessionmaker

In [14]:
Session = sessionmaker(bind=engine) #Bisogna collegare la sessione allo specifico DB e quindi alla specifica connesione ergo all'engine
session = Session()

In [15]:
pkm_1 = Pokemon(1, "Bulbasaur", "Erba", 1)

In [16]:
pkm_2 = Pokemon(2, "Charmender", "Fuoco", 1)

In [17]:
pkm_3 = Pokemon(3, "Squirtle", "Acqua", 1)

In [18]:
session.add_all([pkm_1, pkm_2, pkm_3])

In [19]:
session.commit()

2022-11-24 17:15:43,492 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-24 17:15:43,495 INFO sqlalchemy.engine.Engine INSERT INTO pokemon (nome, tipo, generazione, data_creazione) VALUES (?, ?, ?, ?)
2022-11-24 17:15:43,496 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ('Bulbasaur', 'Erba', 1, '2022-11-24 16:15:43.495831')
2022-11-24 17:15:43,497 INFO sqlalchemy.engine.Engine INSERT INTO pokemon (nome, tipo, generazione, data_creazione) VALUES (?, ?, ?, ?)
2022-11-24 17:15:43,498 INFO sqlalchemy.engine.Engine [cached since 0.002376s ago] ('Charmender', 'Fuoco', 1, '2022-11-24 16:15:43.497788')
2022-11-24 17:15:43,498 INFO sqlalchemy.engine.Engine INSERT INTO pokemon (nome, tipo, generazione, data_creazione) VALUES (?, ?, ?, ?)
2022-11-24 17:15:43,499 INFO sqlalchemy.engine.Engine [cached since 0.003411s ago] ('Squirtle', 'Acqua', 1, '2022-11-24 16:15:43.498819')
2022-11-24 17:15:43,499 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
deb_1 = Debolezze(1, pkm_1.tipo, "Fuoco")

2022-11-24 17:16:37,284 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-24 17:16:37,287 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm AS pokemon_id_pkm, pokemon.nome AS pokemon_nome, pokemon.tipo AS pokemon_tipo, pokemon.generazione AS pokemon_generazione, pokemon.data_creazione AS pokemon_data_creazione 
FROM pokemon 
WHERE pokemon.id_pkm = ?
2022-11-24 17:16:37,287 INFO sqlalchemy.engine.Engine [generated in 0.00049s] (1,)


In [24]:
deb_2 = Debolezze(2, pkm_2.tipo, "Acqua")

2022-11-24 17:16:38,015 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm AS pokemon_id_pkm, pokemon.nome AS pokemon_nome, pokemon.tipo AS pokemon_tipo, pokemon.generazione AS pokemon_generazione, pokemon.data_creazione AS pokemon_data_creazione 
FROM pokemon 
WHERE pokemon.id_pkm = ?
2022-11-24 17:16:38,016 INFO sqlalchemy.engine.Engine [cached since 0.7298s ago] (2,)


In [25]:
deb_3 = Debolezze(3, pkm_3.tipo, "Erba")

2022-11-24 17:16:38,662 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm AS pokemon_id_pkm, pokemon.nome AS pokemon_nome, pokemon.tipo AS pokemon_tipo, pokemon.generazione AS pokemon_generazione, pokemon.data_creazione AS pokemon_data_creazione 
FROM pokemon 
WHERE pokemon.id_pkm = ?
2022-11-24 17:16:38,662 INFO sqlalchemy.engine.Engine [cached since 1.376s ago] (3,)


In [26]:
session.add_all([deb_1, deb_2, deb_3])

In [27]:
session.commit()

2022-11-24 17:16:41,597 INFO sqlalchemy.engine.Engine INSERT INTO debolezze (tipo, debolezza) VALUES (?, ?)
2022-11-24 17:16:41,598 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ('Erba', 'Fuoco')
2022-11-24 17:16:41,599 INFO sqlalchemy.engine.Engine INSERT INTO debolezze (tipo, debolezza) VALUES (?, ?)
2022-11-24 17:16:41,599 INFO sqlalchemy.engine.Engine [cached since 0.002038s ago] ('Fuoco', 'Acqua')
2022-11-24 17:16:41,600 INFO sqlalchemy.engine.Engine INSERT INTO debolezze (tipo, debolezza) VALUES (?, ?)
2022-11-24 17:16:41,600 INFO sqlalchemy.engine.Engine [cached since 0.002903s ago] ('Acqua', 'Erba')
2022-11-24 17:16:41,601 INFO sqlalchemy.engine.Engine COMMIT


## Interrogare il DB:
Per interrogare il DB possiamo usare i comandi di SQLAlchemy indipendti dal dialetto:
sqlAlchemy.query()
oppure definire una query ed eseguirla con session.execut(qury)

### SELECT  *
COME OTTENERE TUTTI I DATI DI UNA TABELLA:

In [28]:
all_pokemon = session.query(Pokemon)

In [29]:
for pokemon in all_pokemon:
    print(pokemon)
    print()

2022-11-24 17:16:44,032 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-24 17:16:44,033 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm AS pokemon_id_pkm, pokemon.nome AS pokemon_nome, pokemon.tipo AS pokemon_tipo, pokemon.generazione AS pokemon_generazione, pokemon.data_creazione AS pokemon_data_creazione 
FROM pokemon
2022-11-24 17:16:44,033 INFO sqlalchemy.engine.Engine [generated in 0.00046s] ()
(1) Bulbasaur, Erba, 1)

(2) Charmender, Fuoco, 1)

(3) Squirtle, Acqua, 1)



Equilvalentemente si può interrogare il DB con una struttura che rispecchia il linguaggio SQL: \
NB: usiamo il context manager *with()*

In [30]:
query = sal.select([Pokemon])

In [31]:
with engine.connect() as conn:
    r = conn.execute(query).fetchall()

2022-11-24 17:16:47,630 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm, pokemon.nome, pokemon.tipo, pokemon.generazione, pokemon.data_creazione 
FROM pokemon
2022-11-24 17:16:47,631 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ()


In [32]:
for result in list(r):
    print(result)

(1, 'Bulbasaur', 'Erba', 1, datetime.datetime(2022, 11, 24, 16, 15, 43, 495831))
(2, 'Charmender', 'Fuoco', 1, datetime.datetime(2022, 11, 24, 16, 15, 43, 497788))
(3, 'Squirtle', 'Acqua', 1, datetime.datetime(2022, 11, 24, 16, 15, 43, 498819))


### ORDER BY:

In [33]:
ordered_pokemon = session.query(Pokemon).order_by(Pokemon.nome)

In [34]:
for pokemon in ordered_pokemon:
    print(pokemon.nome)

2022-11-24 17:16:50,658 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm AS pokemon_id_pkm, pokemon.nome AS pokemon_nome, pokemon.tipo AS pokemon_tipo, pokemon.generazione AS pokemon_generazione, pokemon.data_creazione AS pokemon_data_creazione 
FROM pokemon ORDER BY pokemon.nome
2022-11-24 17:16:50,659 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ()
Bulbasaur
Charmender
Squirtle


EQUIVALENTE SQL :
```SQL
SELECT nome 
FROM pokemon
ORDER BY nome DESC,
```

In [35]:
query = sal.select([Pokemon]).order_by(sal.desc(Pokemon.nome))

ordered_pokemon = session.execute(query)
for pokemon in ordered_pokemon:
    print(pokemon)

2022-11-24 17:16:52,457 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm, pokemon.nome, pokemon.tipo, pokemon.generazione, pokemon.data_creazione 
FROM pokemon ORDER BY pokemon.nome DESC
2022-11-24 17:16:52,458 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ()
((3) Squirtle, Acqua, 1),)
((2) Charmender, Fuoco, 1),)
((1) Bulbasaur, Erba, 1),)


### WHERE:
EQUIVALENTE SQL :
```SQL
SELECT * 
FROM pokemon
WHERE generazione = 1,
```

In [36]:
query = sal.select([Pokemon]).where(Pokemon.generazione == 1)
result = session.execute(query)

2022-11-24 17:16:54,236 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm, pokemon.nome, pokemon.tipo, pokemon.generazione, pokemon.data_creazione 
FROM pokemon 
WHERE pokemon.generazione = ?
2022-11-24 17:16:54,237 INFO sqlalchemy.engine.Engine [generated in 0.00114s] (1,)


In [37]:
for r in result:
    print(r)

((1) Bulbasaur, Erba, 1),)
((2) Charmender, Fuoco, 1),)
((3) Squirtle, Acqua, 1),)


### IN:
EQUIVALENTE SQL :
```SQL
SELECT nome, tipo
FROM pokemon
WHERE generazione IN (1, 2),
```

In [38]:
query = sal.select([Pokemon.nome, Pokemon.tipo]).where(Pokemon.generazione.in_([1, 2]))

In [39]:
result = session.execute(query)

2022-11-24 17:16:57,501 INFO sqlalchemy.engine.Engine SELECT pokemon.nome, pokemon.tipo 
FROM pokemon 
WHERE pokemon.generazione IN (?, ?)
2022-11-24 17:16:57,502 INFO sqlalchemy.engine.Engine [generated in 0.00103s] (1, 2)


In [40]:
for r in result:
    print(r)

('Bulbasaur', 'Erba')
('Charmender', 'Fuoco')
('Squirtle', 'Acqua')


### LIKE:
EQUIVALENTE SQL :
```SQL
SELECT nome,
FROM pokemon
WHERE nome LIKE '%saur',
```

In [43]:
results = session.query(Pokemon).filter(Pokemon.nome.like("%saur"))

In [46]:
print(results)

SELECT pokemon.id_pkm AS pokemon_id_pkm, pokemon.nome AS pokemon_nome, pokemon.tipo AS pokemon_tipo, pokemon.generazione AS pokemon_generazione, pokemon.data_creazione AS pokemon_data_creazione 
FROM pokemon 
WHERE pokemon.nome LIKE ?


In [50]:
for r in results:
    print(r)

2022-11-24 17:20:09,827 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm AS pokemon_id_pkm, pokemon.nome AS pokemon_nome, pokemon.tipo AS pokemon_tipo, pokemon.generazione AS pokemon_generazione, pokemon.data_creazione AS pokemon_data_creazione 
FROM pokemon 
WHERE pokemon.nome LIKE ?
2022-11-24 17:20:09,829 INFO sqlalchemy.engine.Engine [cached since 17.31s ago] ('%saur',)
(1) Bulbasaur, Erba, 1)


### Convertire query in DataFrame:

In [63]:
df = pd.read_sql(session.query(Pokemon).statement,session.bind) 
df

2022-11-24 17:34:15,544 INFO sqlalchemy.engine.Engine SELECT pokemon.id_pkm, pokemon.nome, pokemon.tipo, pokemon.generazione, pokemon.data_creazione 
FROM pokemon
2022-11-24 17:34:15,545 INFO sqlalchemy.engine.Engine [cached since 42.42s ago] ()


,id_pkm,nome,tipo,generazione,data_creazione
0,1,Bulbasaur,Erba,1,2022-11-24 16:15:43.495831
1,2,Charmender,Fuoco,1,2022-11-24 16:15:43.497788
2,3,Squirtle,Acqua,1,2022-11-24 16:15:43.498819


_______________________